# Funktionen: Gültigkeitsbereich von Variablen

Bisher haben wir uns noch keine großen Gedanken darüber gemacht, wann und wo der Wert einer Variable sichtbar ist. In Zusammenhang mit Funktionen müssen wir uns jedoch damit beschäftigen. Vorauszuschicken ist, dass diese Sichtbarkeit in Python eher ungewöhnlich gelöst ist.

## Scopes (Gültigkeitsbereiche)

Grundsätzlich müssen wir zwei Gültigkeitsbereich unterscheiden:

  * Den globalen Gültigkeitsbereich
    * Jede im Hauptprogramm (also außerhalb einer Funktion) angelegt Variable 
      lebt im globalen Scope.
    * Globale Variablen sind auch innerhalb einer Funktion sichtbar
  * Den lokalen Gültigkeitsbereich
    * Jede innerhalb einer Funktion angelegte Variable ist nur innerhalb der 
      Funktion sichtbar (außer sie ist mit dem Keyword ``global`` als global 
      definiert, was aber fast immer eine schlechte Idee ist, weshalb Sie das 
      gleich wieder vergessen sollten).
      
Probieren wir die Gültigkeitsbereiche gleich aus. Im folgenden Beispiel legen wir eine globale Variable ``val`` an, die dann in der Funktion verwendet wird:      

In [ ]:
def a_function():
    print(val)
    
val = 1
a_function()

Wie wir sehen, ist die außerhalb der Funktion angelegte Variable ``val`` auch innerhalb der Funktion sichtbar: Wir können ihren Wert ausgeben.

Anders herum funktioniert es nicht: Eine in einer Funktion angelegte **lokale Variable** ist außerhalb der Funktion (d.h. im globalen Scope) nicht sichtbar:

In [ ]:
def a_function():
    other_val = 1
    
a_function()
other_val

Hier noch eine Anmerkung zum ersten, funktionierenden Beispiel: Es ist in den meisten Fällen unschön, auf diese Weise das Vorhandensein einer globalen Variable vorauszusetzen. Außerdem leidet die Nachvollziehbarkeit des Codes,  weil man nicht nur die Funktion verstehen, sondern auch herausfinden muss, was es mit der globalen Variable auf sich hat.

Besser ist, die in der Funktion benötigten Werte explizit via Parameter an die Funktion zu übergeben. Dann sieht man sofort, welche Werte die Funktion erwartet und zusätzlich kann man die Funktion einfacher testen. Das oben stehende Beispiel sollte also besser so geschrieben werden:

In [ ]:
def a_function(value):
    print(value)
    
val = 1
a_function(val)

In diesem Falle ist ``value`` eine lokale, ``val`` eine globale Variable. Die beiden sind also einfach auseinander zu halten, weil wir unterschiedliche Namen verwendet haben. Im Prinzip kann man aber als Parametername (sprich: lokale Variable) denselben Namen verwenden wie für die globale Variable:

In [ ]:
def a_function(val):
    print(val)
    
val = 1
a_function(val)

Wie wir sehen, funktioniert auch diese Lösung. Es ist aber wichtig zu verstehen, dass wir es mit zwei unterschiedlichen Variablen zu tun haben, auch wenn sie denselben Namen tragen. Gleichnamige Variablen im globalen und lokalen Scope sind also grundsätzlich keine Problem. Die Lösung mit unterschiedlichen Variablennamen ist aber meiner Meinung nach einfacher nachvollziehbar.

## Parameterwerte werden als Referenz übergeben

Kommen wir nun zu einer Besonderheit, die schon häufig dazu geführt hat, dass Python Programme nicht sauber funktionieren. Der Grund dafür liegt daran, dass Python versucht, Objekte nur dann zu kopieren, wenn es unumgänglich ist. Man spricht hier von *shallow copies* also "seichten" Kopien. 

Um das zu verstehen, lassen wir uns nun mit der ``id()`` Funktion anzeigen, auf welche Objekte die beiden  Variablen zeigen:

In [ ]:
def a_function(value):
    print(id(value))
    
val = 1
print(id(val))
a_function(val)

Zu unserer Verwirrung sehen wir, dass beide Variablen (also die globale und die lokale) auf dasselbe Objekt zeigen. Haben wir es also doch nicht mit zwei unterschiedlichen Variablen und Gültigkeitsbereichen zu tun? 

Um dieses Verhalten zu verstehen, müssen wir wissen, dass Python aus Effizienzgründen für Funktionsparameter nie volle Kopien ("Deep Copies") des übergebenen Wertes erzeugt, sondern eine "Shallow Copy" verwendet. Diese enthält, so lange der Wert sich nicht ändert, einfach eine Referenz auf das übergebene Originalobjekt. Die Id der globalen und der lokalen Variable ist zunächst einmal gleich, weil beide sich auf dasselbe Objekt im Hauptspeicher beziehen.

Erst, wenn wir innerhalb der Funktion den übergebenen Wert verändern, wird ein neues Objekt (im lokalen Scope) erzeugt.

In [ ]:
def increase(val):
    print(f"[b]: id von val in der Funktion (vor der Änderung): {id(val)}")
    val += 1
    print(f"[c]: id von val in der Funktion nach der Änderung: {id(val)}")

val = 1
print(f"[a]: id von val außerhalb der Funktion: {id(val)}")

increase(val)
print(f"[d]: id von val außerhalb der Funktion: {id(val)}")

Die globale Variable (in der Ausgabe [a] und [d], im Code Zeilen 7 und 10) zeigt immer auf dasselbe Objekt.
Innerhalb der Funktion zeigt ``val`` zuerst ebenfalls auf dieses Objekt (Ausgabe [b] bzw. Zeile 2). Erst nachdem wir in der Funktion den Wert der lokalen Variable verändert haben, zeigt diese auf ein anderes Objekt ([b] bzw. Zeile 4). Dass dies nur für den lokalen Gültigkeitsbereich gilt, sehen wir daran, dass sich die id der globalen Variable nicht geändert hat ([d] bzw. Zeile 10). Fassen wir also zusammen: Lokale Variablen innerhalb einer Funktion sind so lange Referenzen auf das global referenziert Objekt, solange wir innerhalb der Funktion diesen Wert nicht verändern.

Eigentlich sollte uns dieses Verhalten nicht überraschen, weil wir bereits gelernt haben, dass ``int`` wie auch einige andere Datentypen zu den unveränderbaren Datentypen gehören: Jedes Mal, wenn wir den Wert verändern, wird ein neues Objekt angelegt. In unserem Fall ist eigentlich nur neu, dass das nach der Änderung angelegt Objekt nur im lokalen Scope sichtbar ist.

## Scopes veränderbarer Datentypen

Was aber passiert, wenn wir veränderbare Datentypen (z.B. Listen oder Dictionaries) an die Funktion übergeben (bzw. nicht  übergebene globale Variablen verwenden)? Probieren wir es aus, indem wir eine Liste an eine Funktion übergeben und dann in der Funktion ein Element der Liste ersetzen:

In [ ]:
def compute_final_grade(grades):
    print(f"(b) {grades} {id(grades)}")
    grades[1] = 1
    print(f"(c) {grades} {id(grades)}")
    
grades = [3, 5, 4, 1]    
print(f"(a) {grades} {id(grades)}")
compute_final_grade(grades)
print(f"(d) {grades} {id(grades)}")

Wie wir sehen, ist die Liste ``grades`` innerhalb der Funktion dasselbe Objekt (mit derselben id) wie außerhalb, auch wenn wir einen Wert verändern (Zeile 3). Mit anderen Worten: Wenn wir einen veränderbaren Datentyp innerhalb einer Funktion verändern, erzeugen wir kein neues Objekt, sondern wir verändern das Objekt aus dem globalen Gültigkeitbereich! 

Dieses Verhalten kann schnell zu unbeabsichtigten Nebeneffekten und damit zu Fehlern führen, die sehr schwer zu finden sind. Man kann sich am einfachsten dagegen schützen, indem man 

   * keine veränderbaren Typen (Listen, Dictionaries, ...) als Argumente an eine Funktion übergibt
   * oder zumindest penibel darauf achtet, dass der übergebene Wert innerhalb der 
     Funktion nicht verändert wird 
   * Alternativ kann man mit Kopien oder Typänderungen auf nicht
     veränderbare Typen (wie Tupel) arbeiten. Dabei ist aber zu bedenken, dass durch den 
     Kopiervorgang mitunter ein erheblicher Overhead entsteht.

### Exkurs: Shallow Copy und Deep Copy

Falls Sie sich mit dem Gedanken tragen, aus Sicherheitsgründen in der Funktion (oder schon vorher) für veränderbare Datentypen eine Kopie zu erzeugen, sollten Sie unbedingt die folgenden Ausführungen lesen, damit Sie keine böse Überraschung erleben.

Bei der Erstellung einer Kopie ist zu beachten, dass bei Elementen mit veränderbarem Datentyp unerwartete Probleme auftreten können. Bleiben wir zunächst bei den unveränderbaren Typen. Im folgenden Codebeispiel erzeugen wir in der Funktion eine Kopie der Liste ``data`` und geben dieser den Namen ``ldata``:

In [45]:
def a_function(data):
    ldata = data[:]
    
    print(f"data: {id(data)}")
    print(f"ldata: {id(ldata)}")
    print(f"data[0]: {id(data[0])}")
    print(f"ldata[0]: {id(ldata[0])}")
    ldata[0] = "Z"
    print(f"ldata[0] nach Änderung: {id(ldata[0])}")
    print(f"data[0] nach Änderung: {id(data[0])}")
    
a_function(["A", "B", "C"])    

data: 137199178011072
ldata: 137199177991360
data[0]: 137199544640048
ldata[0]: 137199544640048
ldata[0] nach Änderung: 137199535523696
data[0] nach Änderung: 137199544640048


Hier sehen wir, dass das erste Element der Kopie der Liste (also ``ldata[0]``) zunächst auf dasselbe Objekt zeigt wie das erste Objekt der originalen Liste ``data``. Erst wenn wir den Wert von ``ldata[0]`` ändern, ändert sich auch die Id des Objekts. Das wurde bewußt so designed, weil dadurch nicht benötigte doppelte Werte vermieden werden ("shallow copy"). Bei unveränderbaren Werten funktioniert das auch problemlos, weil diese ohnehin neu erzeugt werden. Wir sehen, dass die Änderung von ``ldata[0]`` den Wert von ``data[0]`` nicht verändert hat.

Wir müssen aber im Hinterkopf behalten, dass das Kopieren von Containern anders funktioniert, wenn die enthaltenen Objekte zu den veränderbaren Datentypen gehören. Im folgenden Beispiel haben wir es nicht mehr mit einer Liste von Strings, sondern mit einer Liste von Listen zu tun. Wir machen in der Funktion eine Kopie der Liste und verändern in der Funktion den ersten Wert des ersten Listenelements der kopierten Liste:

In [46]:
def a_function(data):
    ldata = data[:]
    
    print(f"data: {id(data)}")
    print(f"ldata: {id(ldata)}")
    print(f"data[0]: {id(data[0])}")
    print(f"ldata[0]: {id(ldata[0])}")
    ldata[0][0] = "Z"
    print(f"data[0] nach Änderung: {id(data[0])}")
    print(f"ldata[0] nach Änderung: {id(ldata[0])}")
    
a_function([["A", "B", "C"], ["D", "E", "F"]])    

data: 137199177838336
ldata: 137199179710144
data[0]: 137199177737984
ldata[0]: 137199177737984
data[0] nach Änderung: 137199177737984
ldata[0] nach Änderung: 137199177737984


Hier sehen wir, dass die Änderung von ``ldata[0]`` auch ``data[0]`` verändert hat. Der Grund dafür liegt in der oben beschrieben Art und Weise, wie Python beim Kopieren von Containerobjekten vorgeht: Kopien von Werten werden erst erzeugt, wenn ein neues Objekt angelegt wird. Da bei veränderbaren Objekten kein neues Objekt erzeugt wird, wird nicht kopiert. Dadurch verändern wir nicht nur den Wert in der Kopie, sondern auch im Original! Die einzige Möglichkeit, dies zu verhindern, liegt darin, eine "tiefe Kopie" (deep copy) des Originalcontainers zu erzeugen. Dabei wird nicht nur der Container kopiert, sondern auch sein gesamter Inhalt. Hier ein Beispiel:

In [ ]:
import copy


def a_function(data):
    ldata = copy.deepcopy(data)
    
    print(f"data: {id(data)}")
    print(f"ldata: {id(ldata)}")
    print(f"data[0]: {id(data[0])}")
    print(f"ldata[0]: {id(ldata[0])}")
    ldata[0][0] = "Z"
    print(f"data[0] nach Änderung: {id(data[0])}")
    print(f"ldata[0] nach Änderung: {id(ldata[0])}")    
    
a_function([["A", "B", "C"], ["D", "E", "F"]])   

Hier haben wir zum Kopieren die ``copy.deepcopy()`` Funktion verwendet. Damit ist nicht nur der Container ``ldata`` eine Kopie von ``data``, sondern auch alle darin enthaltenen Objekte sind Kopien der Originalwerte. Wir können also nun Werte in ``ldata`` verändern, ohne dass dadurch die Werte in ``data`` tangiert werden. Es muss uns allerdings klar sein, dass eine solche tiefe Kopieraktion bei größeren Datenmengen "teuer" im Sinne von Rechenzeit und Speicherbedarf werden kann.

Mit diesem Wissen sollte deutlicher werden, warum man auf die Übergabe von veränderbaren Typen als Funktionsargumente verzichten sollte.

## Vertiefende Literatur
Ich empfehle ausdrücklich, mindestens eine der folgenden Ressourcen zur Vertiefung zu lesen!

  * Python Tutorial: 
	* Kapitel 4.6 - Defining Functions 
      (https://docs.python.org/3/tutorial/controlflow.html#defining-functions)
    * Kapitel 4.7 - More on Defining Functions
	  (https://docs.python.org/3/tutorial/controlflow.html#more-on-defining-functions)
  * Klein, Kurs: 
	* Funktionen (https://python-kurs.eu/python3_funktionen.php)
	* Parameter-Übergabe (http://python-kurs.eu/python3_parameter.php)
	* Globale und lokale Variablen (http://python-kurs.eu/python3_global_lokal.php)
	* Rekursive Funktionen (http://python-kurs.eu/python3_rekursive_funktionen.php)
	* Flaches und tiefes Kopieren (http://python-kurs.eu/python3_deep_copy.php)
  * Sweigart: https://automatetheboringstuff.com/2e/chapter3/  
    
    
  * Klein, Buch: Kapitel 14, 15 und evtl. 13.
  * Kofler: Kapitel 9.
  * Inden: Kapitel 2.5.
  * Weigend: Kapitel 6.1 bis 6.8 und 6.14.
  * Pilgrim: Kapitel 1.2
    (https://www.diveinto.org/python3/your-first-python-program.html#declaringfunctions)
  * Downey: Kapitel 3
    (http://www.greenteapress.com/thinkpython/html/thinkpython004.html)
    
    
  * Video: Ned Batchelder - Facts and Myths about Python names and values - PyCon 
    2015 (https://www.youtube.com/watch?v=_AEJHKGk9ns)